In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import torch


# Generar datos 

In [ ]:
#generar una línea recta con ruido
data_x=np.arange(-5,5,0.1,dtype=float)
(n_x) = data_x.size
in_w1 = 1.5
in_b = 1.33
ruido = np.random.randn(n_x)
data_y = data_x*in_w1 + in_b #*ruido

In [ ]:
#graficar
plt.plot(data_x,data_y)

In [ ]:
plt.scatter(data_x,data_y)

In [ ]:
#guardar al archivo
salida = pd.DataFrame( )
salida['feature0'] = pd.DataFrame(data_x)
salida['label'] = pd.DataFrame(data_y)
salida.set_index('feature0')

In [ ]:
salida.head()

In [ ]:
salida.info()

In [ ]:
salida.to_csv("sample.csv", index=False)

In [ ]:
#subir al archivo sample.csv a Google Drive
!cat sample.csv

# Ingestar Datos

In [ ]:
#dataset=pd.read_csv('https://uc74727dc6f1c80cfcaaea78b73c.dl.dropboxusercontent.com/cd/0/inline/B1nnXKCSKQUkDeh0zCe4HVTbxwKDQQcIhWeSR0b4aH4r08KvF52jYt_hM5fw1nCsuA_xIFl9g7b2L7zxBO3I8OUffVoDPjTichkALgvk4DUF6id5nPw2pohzFtcc3Ouq6eNSPaSuP9Ta2jRAl5zwTn4hXZPhJoITY2J_5S2DDT1dgg/file#;')
dataset = pd.read_csv("sample.csv")
    


In [ ]:
dataset.head(20)

In [ ]:
plt.plot(dataset['feature0'],dataset['label'])

In [ ]:
from sklearn.model_selection import train_test_split



In [ ]:
(dataset_feature0_train, dataset_feature0_test,dataset_label_train, dataset_label_test )= train_test_split(dataset['feature0'],dataset['label'],test_size=0.3,train_size=0.7)

# Entrenar

In [ ]:
def estimate(x,w_1,b_1) :
  return w_1*x + b_1

In [ ]:
def loss(y,y_fit):
  return torch.mean((y - y_fit)**2)

In [ ]:
def train(features,target,etha,debug=False,iter=100) :
   loss_list = []

   w_1 = torch.randn(1, requires_grad=True)
   b_1 = torch.randn(1, requires_grad=True)

   
   for i in range(iter):
     if debug :
       print( i,(w_1.item(),b_1.item()))
     
     y_fit = estimate(features,w_1,b_1)
     if debug:
       print( i,y_fit)
     L=loss(y_fit,target)
     loss_list.append(L.item())
     L.backward() 
     if debug:
      print(L.item())
      print((w_1.grad.item(),b_1.grad.item()))
    # with tr.no_grad():
     w_1.data = w_1.data -  w_1.grad.data*etha
     b_1.data = b_1.data -  b_1.grad.data*etha
     w_1.grad.zero_()
     b_1.grad.zero_()
   return (w_1.item(),b_1.item(),loss_list) 
    

In [ ]:
x_feature = torch.from_numpy(  dataset_feature0_train.to_numpy() )
y_label = torch.from_numpy(  dataset_label_train.to_numpy() )

In [ ]:
(e_w_1,e_b_1,e_L)=train(x_feature,y_label,0.00001,debug=False,iter=100000)

In [ ]:
plt.xlabel("Iteracion")
plt.ylabel("Loss")
plt.plot(e_L,'r')

In [ ]:
plt.plot(x_feature,y_label,"g")
plt.plot(x_feature,estimate(x_feature,e_w_1,e_b_1),"r")

In [ ]:
def MSE_score( y_observada , y_estimada):
   return ((y_observada - y_estimada)**2).mean().item()

In [ ]:
def R2_score( y_observada , y_estimada):
    numerador = ((y_observada - y_estimada)**2).sum()
    y_promedio = y_observada.mean()
    denominador = ((y_observada - y_promedio)**2).sum()
    return (1 - (numerador/denominador)).item()


In [ ]:
MSE_score(y_label,estimate(x_feature,e_w_1,e_b_1))

In [ ]:
R2_score(y_label,estimate(x_feature,e_w_1,e_b_1))

In [ ]:
print((e_w_1,e_b_1))

In [ ]:
#escribir los parametros del modelo a un archivo
import pickle
ml_params= {
    "w_1" : e_w_1,
    "b" : e_b_1
}

pickle.dump(ml_params,open("mlparams", 'wb') )

In [ ]:
!cat mlparams

In [ ]:
!od -x mlparams

# Validar

In [ ]:
import pickle
ml_param_validate=pickle.load(open("mlparams", 'rb'))

In [ ]:
x_test = torch.from_numpy(  dataset_feature0_test.to_numpy() )
y_fit_test= estimate(x_test,ml_param_validate['w_1'],ml_param_validate['b'])
y_test = torch.from_numpy(  dataset_label_test.to_numpy() )

In [ ]:
plt.plot(x_test,y_fit_test)
plt.plot(x_test,y_test)

In [ ]:
def MSE_score( y_observada , y_estimada):
   return ((y_observada - y_estimada)**2).mean().item()

In [ ]:
def R2_score( y_observada , y_estimada):
    numerador = ((y_observada - y_estimada)**2).sum()
    y_promedio = y_observada.mean()
    denominador = ((y_observada - y_promedio)**2).sum()
    return (1 - (numerador/denominador)).item()


In [ ]:
R2_score(y_test,y_fit_test)

In [ ]:
MSE_score(y_test,y_fit_test)

# Aprovisionar

In [ ]:
!pip install boto3

In [ ]:
import boto3
import os

In [ ]:
os.environ['AWS_ACCESS_KEY_ID']= 'XXXXXX'
os.environ['AWS_SECRET_ACCESS_KEY']= 'XXXXXX'
os.environ ['AWS_DEFAULT_REGION']=  'us-east-1'

In [ ]:
ecr_cliente = client = boto3.client('ecr')

In [ ]:
#crear repositorio para imagen a entrenar
client.create_repository(repositoryName='dummylinear')

In [ ]:
ecs_cliente = boto3.client('ecs')


In [ ]:
ecs_cliente.create_cluster( clusterName='dummylinear')

In [ ]:
ecs_cliente.register_task_definition( 
     family='tskdummylinear2',
     cpu ='256',
     memory = '512',
     requiresCompatibilities= ['FARGATE'],
     runtimePlatform = {'cpuArchitecture': 'X86_64', 'operatingSystemFamily': 'LINUX'},
     networkMode = 'awsvpc' ,
     executionRoleArn = 'arn:aws:iam::740327929864:role/ecsTaskExecutionRole',
     containerDefinitions=[
         { 
            'name': 'dummylinear',
            'image': '740327929864.dkr.ecr.us-east-1.amazonaws.com/dummylinear',
            'portMappings': [{'containerPort': 5000, 'hostPort': 5000, 'protocol': 'tcp', 'name': 'dummylinear-5000-tcp',
                              'appProtocol': 'http'}] ,
         }
     ] ,
    
 )

In [ ]:
ecs_cliente.create_service(cluster='dummylinear',
                           serviceName='svcdummylinear2',
                           taskDefinition='arn:aws:ecs:us-east-1:740327929864:task-definition/tskdummylinear:1',
                           launchType ='FARGATE',
                           desiredCount=1,
                           networkConfiguration= {
                               'awsvpcConfiguration': {
                                  'assignPublicIp' : 'ENABLED',
                                  'securityGroups': ['sg-08ec75aa8b8f7c45b'],
                                  'subnets': ['subnet-0d5bef679df57ba9e',
                                               'subnet-02efe5c7251ec5212',
                                               'subnet-02acd8f884e171a3c',
                                               'subnet-08c84ad9ae30a370d',
                                               'subnet-0615686222a56563b',
                                               'subnet-01c5e49442af85ab1'],
                                }   
                            }
                          )

In [ ]:
ecs_cliente.describe_task_definition( taskDefinition='tskdummylinear2') 

In [ ]:
ecs_cliente.describe_services(cluster='dummylinear',services=["svcdummylinear"])

# Inferir

In [ ]:
import pickle

In [ ]:
api_mlparams=pickle.load(open("mlparams", 'rb'))

In [ ]:
!pip install flask==0.12.2

In [ ]:
from flask import Flask
from flask import request
import pickle
import logging
import sys
def predict(x,w_1,b_1) :
  return w_1*x + b_1
  
print(__name__)
app = Flask(__name__)
api_mlparams=pickle.load(open("mlparams", 'rb'))

logging.info(api_mlparams)

@app.route('/infer')
def infer():
   w_1 = api_mlparams['w_1']
   b = api_mlparams['b']
   reqX = request.args.get('x')
   x = float(reqX)
   print((x,type(x)),file=sys.stderr)
   return {'y':w_1*x  + b}

if __name__ == '__main__':
    app.run(host='0.0.0.0',debug=False)